# Solving MNIST with ANN

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [57]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
mnist=fetch_mldata('MNIST original')
stdsc=StandardScaler()
#X,y=pd.DataFrame(mnist['data']), pd.DataFrame(mnist['target'])
X,y=mnist['data'], mnist['target'].astype(int)
X=stdsc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

/home/igor/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [59]:
mnist['target'],y

(array([0., 0., 0., ..., 9., 9., 9.]), array([0, 0, 0, ..., 9, 9, 9]))

In [125]:
#feature_cols=[tf.feature_column.numeric_column(key=key) for key in X_train.columns]
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnnc=tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10, feature_columns=feature_cols)
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=2, batch_size=50, shuffle=True)
dnnc.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp99t_jqmi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe0715a20b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp99t_jqmi/model.ckpt.
INFO:tensorflow:loss = 167.5

In [84]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnnc.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-25-13:57:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1m215olj/model.ckpt-2240
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-25-13:57:24
INFO:tensorflow:Saving dict for global step 2240: accuracy = 0.9685714, average_loss = 0.112394795, global_step = 2240, loss = 14.304791


In [117]:
preds=dnnc.predict(input_fn=test_input_fn)
pred_results=list(preds)
pred_results[1]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1m215olj/model.ckpt-2240
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([-4.4332805, -5.39123  ,  5.754981 ,  6.6078234, -4.0162997,
         1.15252  , -6.320815 , -2.5382886, 16.437817 ,  1.7385228],
       dtype=float32),
 'probabilities': array([8.6250951e-10, 3.3092687e-10, 2.2933473e-05, 5.3808959e-05,
        1.3087489e-09, 2.2995692e-07, 1.3062246e-10, 5.7378449e-09,
        9.9992263e-01, 4.1318455e-07], dtype=float32),
 'class_ids': array([8]),
 'classes': array([b'8'], dtype=object)}

In [118]:
y_test_preds=np.array([p['class_ids'][0] for p in pred_results])

In [122]:
np.mean(y_test==y_test_preds)

0.9685714285714285